Read input into a data structure

In [1]:
from copy import deepcopy
from collections import deque

def read_input(path: str) -> list:
    monkeys = []
    monkey_biz = []
    for l in open(path):
        if l == '\n':
            monkeys.append(deepcopy(monkey_biz))
            monkey_biz.clear()
        else:
            monkey_biz.append(l.strip())
    monkeys.append(monkey_biz.copy())
    del monkey_biz

    # Clean up data; a more polite version of me would make a Monkey class with named attributes
    for i, biz in enumerate(monkeys):
        biz = biz[1:]  # Remove the first line; already encoded in 'monkeys' via index
        biz[0] = deque([int(_) for _ in biz[0].split(': ')[1].split(', ')])  # Extract item list
        biz[1] = tuple(biz[1][21:].split())  # Extract operator and operand
        biz[2] = int(biz[2][19:]) # Extract dividend for floor operation
        biz[3] = int(biz[3][25:]) # Extract monkey index for 'true' condition
        biz[4] = int(biz[4][25:]) # Extract monkey index for 'false' condition
        biz.append(0)  # Add an entry for number of items inspected by this monkey
        monkeys[i] = biz

    return monkeys


Part 1: 20 rounds, worrying generates relief

In [7]:
def do_business(monkeys: list, rounds: int):
    """
    One "round":
    0. For each monkey:
        1. For each starting value:
            2. Monkey.inspect ("operation" on next in FIFO)
            3. Me.relief (value // 3)
            4. Monkey.test ("test")
            5. Monkey.throw(bool)
            6. OtherMonkey.catch (append to FIFO)
    """
    for _ in range(rounds):
        for m, biz in enumerate(monkeys):
            for instr in range(len(biz[0])):  # Step through the list of instructions
                try:
                    item_val = biz[0].popleft()
                    biz[5] += 1
                except IndexError as e:
                    print(f'Monkey {m}, instruction {instr}: {e}')
                    continue

                operand = biz[1][1]
                match biz[1][0]:
                    case '+':
                        item_val += int(operand)
                    case '-':
                        item_val -= int(operand)
                    case '*':
                        if operand == 'old':
                            item_val = item_val ** 2
                        else:
                            item_val *= int(operand)
                    case '/':
                        item_val /= int(operand)

                item_val = item_val // 3
                throw_to = biz[3] if not item_val % biz[2] else biz[4]
                monkeys[throw_to][0].append(item_val)


monkeys = read_input('../inputs/day11-input')
do_business(monkeys, 20)
inspections = [biz[5] for biz in monkeys]
print(inspections)
inspections.sort(reverse=True)
print(f'Solution: {inspections[0]} * {inspections[1]} = {inspections[0] * inspections[1]}')

[32, 337, 342, 60, 253, 339, 352, 17]
Solution: 352 * 342 = 120384


Part 2: 10k rounds, Chinese Remainder Theorem manages int size

In [4]:
# Try to flush the data structure from the kernel's memory so we don't accidentally reuse it
import gc
try:
    monkeys
    del monkeys
    gc.collect()
    monkeys
    raise RuntimeError('Could not cleanup "monkeys" list before starting Part 2.')
except NameError:
    pass

In [5]:
def optim_business(monkeys: list, rounds: int):
    for _ in range(rounds):
        for m, biz in enumerate(monkeys):
            for instr in range(len(biz[0])):  # Step through the list of instructions
                try:
                    item_val = biz[0].popleft()
                    biz[5] += 1
                except IndexError as e:
                    print(f'Monkey {m}, instruction {instr}: {e}')
                    continue

                operand = biz[1][1]
                match biz[1][0]:
                    case '+':
                        item_val += int(operand)
                    case '-':
                        item_val -= int(operand)
                    case '*':
                        if operand == 'old':
                            item_val = item_val ** 2
                        else:
                            item_val *= int(operand)
                    case '/':
                        item_val /= int(operand)

                item_val = item_val // 3
                throw_to = biz[3] if not item_val % biz[2] else biz[4]
                monkeys[throw_to][0].append(item_val)


monkeys = read_input('../inputs/day11-input')

%load_ext line_profiler
%lprun -f optim_business optim_business(monkeys, 10)

inspections = [biz[5] for biz in monkeys]
print(inspections)
# most = []
# for i in range(2):
#     most.append(max(inspections))
#     inspections.remove(max(inspections))
# print(f'Solution: {most[0]} * {most[1]} = {most[0] * most[1]}')

[28, 158, 162, 43, 105, 165, 172, 16]
